In [17]:
import os
import pandas as pd
import rasterio

In [24]:
def is_x_constant(raster):
    return raster.xy(0, 0)[0] == raster.xy(raster.height-1, raster.width-1)[0]

def is_y_constant(raster):
    return raster.xy(0, 0)[1] == raster.xy(raster.height-1, raster.width-1)[1]

path = '../data/files_to_fix/'
files = os.listdir(path)
cities = ['aleppo', 'damascus', 'raqqa', 'daraa', 'homs', 'idlib', 'hama', 'deir']
good_files, bad_files = [], []
for city in cities:
    city_files = [file for file in files if city in file and file.endswith('.tif')]
    for file in city_files:
        raster = rasterio.open(path+file)
        x_is_constant, y_is_constant = is_x_constant(raster), is_y_constant(raster)
        
        if x_is_constant or y_is_constant:
            bad_files.append(file)
            #bad_files.append((file, x_is_constant, y_is_constant))
            
        if not x_is_constant and not y_is_constant:
            good_files.append(file)

On these files, either the latitude or the longitude (or both) are equal on the boundaries of the image (we assume it's contant across the entire image). Hence, we cannot use them for linking them with the annotation.

In [25]:
bad_files

[]

On these files, that problem is not present

In [26]:
good_files

['raqqa_2016_07_01_zoom_19.tif',
 'raqqa_2014_03_21_zoom_19.tif',
 'daraa_2014_05_01_zoom_19.tif',
 'daraa_2016_02_25_zoom_19.tif',
 'daraa_2013_11_10_zoom_19.tif',
 'daraa_2016_04_19_zoom_19.tif',
 'homs_2013_10_31_zoom_19.tif',
 'idlib_2014_05_31_zoom_19.tif',
 'idlib_2014_02_07_zoom_19.tif',
 'hama_2016_07_29_zoom_19.tif',
 'hama_2013_10_31_zoom_19.tif',
 'hama_2014_04_03_zoom_19.tif',
 'deir_2014_09_16_zoom_19.tif',
 'deir_2013_10_24_zoom_19.tif']

If images have the same dimensions, we could try to infer the coordinates from those images that work.

In [7]:
files = os.listdir('../data/city_rasters/')
cities = ['aleppo', 'damascus', 'raqqa', 'daraa', 'homs', 'idlib', 'hama', 'deir']
dimensions = []
for city in cities:
    city_files = [file for file in files if city in file]
    for file in city_files:
        raster = rasterio.open(f'../data/city_rasters/{file}')
        dimensions.append({
            'city': city,
            'width': raster.width,
            'height': raster.height,
            'x_is_constant': is_x_constant(raster),
            'y_is_constant': is_y_constant(raster)
        })

In [11]:
pd.DataFrame(dimensions).groupby('city')['height'].unique()

city
aleppo      [25088]
damascus     [8192]
daraa       [12032]
deir        [11264]
hama        [15616]
homs        [18176]
idlib        [8704]
raqqa        [8960]
Name: height, dtype: object

In [12]:
pd.DataFrame(dimensions).groupby('city')['width'].unique()

city
aleppo      [37376]
damascus    [11776]
daraa       [13312]
deir        [12032]
hama        [15616]
homs        [20736]
idlib       [10752]
raqqa       [23552]
Name: width, dtype: object

In [24]:
raster = rasterio.open(f'../../../Downloads/aleppo_2016-03-29_zoom_19.tif')
x_is_constant, y_is_constant = is_x_constant(raster), is_y_constant(raster)

if x_is_constant or y_is_constant:
    bad_files.append((file, x_is_constant, y_is_constant))

if not x_is_constant and not y_is_constant:
    good_files.append(file)

In [25]:
x_is_constant

False

In [26]:
y_is_constant

False

In [7]:
raster.xy(0, 0)

(37.06650048494342, 36.26723974943158)

In [ ]:
raster.xy(1000000, 0)

In [ ]:
# Bmps to tifs

In [32]:
from PIL import Image

Image.MAX_IMAGE_PIXELS = None

path = '../data/files_to_fix/'
files = [file for file in os.listdir(path) if file.endswith('.bmp')]
good_files, bad_files = [], []
for file in files:
    print(file)
    img = Image.open(path+file).convert('RGB')
    path_to_save = path+file.split('.bmp')[0]+'.tiff'
    img.save(path_to_save, format='TIFF', compression=None)
    raster = rasterio.open(path_to_save)
    x_is_constant, y_is_constant = is_x_constant(raster), is_y_constant(raster)
    if x_is_constant or y_is_constant:
        bad_files.append(file)

    if not x_is_constant and not y_is_constant:
        good_files.append(file)

hama_2014_04_03_zoom_19.bmp
raqqa_2014_03_21_zoom_19.bmp
idlib_2014_02_07_zoom_19.bmp
homs_2013_10_31_zoom_19.bmp
deir_2014_09_16_zoom_19.bmp
deir_2013_10_24_zoom_19.bmp
raqqa_2016_07_01_zoom_19.bmp
idlib_2014_05_31_zoom_19.bmp
homs_2016_05_30_zoom_19.bmp


In [34]:
good_files

['hama_2014_04_03_zoom_19.bmp',
 'raqqa_2014_03_21_zoom_19.bmp',
 'idlib_2014_02_07_zoom_19.bmp',
 'homs_2013_10_31_zoom_19.bmp',
 'deir_2014_09_16_zoom_19.bmp',
 'deir_2013_10_24_zoom_19.bmp',
 'raqqa_2016_07_01_zoom_19.bmp',
 'idlib_2014_05_31_zoom_19.bmp',
 'homs_2016_05_30_zoom_19.bmp']

In [35]:
raster.xy(0, 100)

(100.5, 0.5)

In [36]:
raster.xy(100, 0)

(0.5, 100.5)